In [1]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time
import math

In [2]:
hand_cascade = cv2.CascadeClassifier('features.xml')
feed = cv2.VideoCapture(0)

pts = deque(maxlen=32)

In [3]:
def detect_motion(pts):
    return False

In [4]:
while True:
    _, frame = feed.read()
    
    if frame is None:
        print("Failure in reading video feed")
        break
    
    blur = cv2.GaussianBlur(frame,(5,5),0)
    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    hand = hand_cascade.detectMultiScale(gray, 1.3, 5)
    mask = np.zeros(gray.shape, dtype = "uint8")
    
    for (x,y,w,h) in hand: 
        cv2.rectangle(frame,(x,y),(x+w,y+h), (122,122,0), 2)
        pts.append(((x+w)/2, (y+h)/2))
        cv2.rectangle(mask, (x,y),(x+w,y+h),255,-1)
    
    extracted_hand = cv2.bitwise_and(gray,mask)
    extracted_hand = cv2.GaussianBlur(extracted_hand,(7,7),0)
    _, extracted_hand = cv2.threshold(extracted_hand,222,225,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    contours, hierarchy = cv2.findContours(extracted_hand, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(frame, contours, 0, (255,255,0), 3)
    
    
    if(len(pts) == 16):
        if(detect_motion(pts)):
            # do something
            print("Motion detected")
        else:
            # check for number of deformities
            hand = max(contours, key = cv2.contourArea)
            hull = cv2.convexHull(hand, returnPoints=False)
            defects = cv2.convexityDefects(hand, hull)
            
            fingers = 0
            
            if defects is not None:
                
                for i in range(defects.shape[0]):
                    
                    p,q,r,s = defects[i,0]
                    finger1 = tuple(hand[p][0])
                    finger2 = tuple(hand[q][0])
                    dip = tuple(hand[r][0])
                    
                    a = math.sqrt((finger2[0] - finger1[0])**2 + (finger2[1] - finger1[1])**2)
                    b = math.sqrt((dip[0] - finger1[0])**2 + (dip[1] - finger1[1])**2)
                    c = math.sqrt((finger2[0] - dip[0])**2 + (finger2[1] - dip[1])**2)
            
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57.29
                
                    if angle <= 90:
                        fingers = fingers + 1
                print(fingers)
                cv2.putText(frame, str(fingers)+" Fingers", (25, 50), cv2.FONT_HERSHEY_SIMPLEX, 4, 4)
    
    cv2.imshow("frame", frame)
    
    
    key =  cv2.waitKey(30) & 0xff
    if(key==27):
        break
        
        
cv2.destroyAllWindows()
    

3
